# Learning TensorFlow

## Day 1

We want to follow the tutorial at https://www.tensorflow.org/get_started/get_started, and at the same time experiment with jupyter notebooks.

So here it is. Lesson 1.

### Introduction

In [1]:
import tensorflow as tf

That would be the first thing we need to include, after installing TensorFlow (`pip3 install tensorflow`).

We create a computation by defining a graph (a _model_ in tensorflow lingo, as in "computational model"). Its nodes are constants, parameters (_variables_, or "trainable parameters"), variables (_placeholders_) and operations. (Yes, I find the terminology a bit confusing.) Their interconnection defines the graph.

A simple example:

In [2]:
node1 = tf.constant(3.0)  # constant
node2 = tf.constant(4.0)  # constant
node3 = node1 + node2     # computation that links to two other nodes
a = tf.placeholder(dtype=tf.float32)      # variable ("placeholder")
node4 = node3 + a         # another computation

To evaluate a node, we create a _session_. We can then run the model for certain values given to the placeholders, by running the session:

In [3]:
sess = tf.Session()
print(sess.run(node4, {a: [1, 2]}))

[ 8.  9.]


### Training a model

Let's start by creating the graph for a linear model:

In [4]:
# Variables (trainable parameters).
W = tf.Variable([.3], dtype=tf.float64)
b = tf.Variable([-.3], dtype=tf.float64)

# Placeholders (variables).
x = tf.placeholder(dtype=tf.float64)

# Model (just another node, really, that connects the results of the others).
linear_model = W * x + b

Variables need to be initialized explicitely, running first this on the session:

In [5]:
init = tf.global_variables_initializer()
sess.run(init)

To evaluate the model on training data we need to write a _loss function_. In practice, it's a node that, when evaluated, produces a number that is the lower the better.

In [6]:
y = tf.placeholder(dtype=tf.float64)

# Loss function. A node that computes the difference of our "linear model" node and the expeted results node.
loss = tf.reduce_sum(tf.square(linear_model - y))  # sum of the squares

# Training data. When we feed the x values, we expect our model to produce something similar to these y values.
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

We would like that, when evaluating the loss node on the training data, the result would be small:

In [7]:
print(sess.run(loss, {x: x_train, y: y_train}))

23.66


Not exciting. We will continue from here.

## Day 2

Instead of diving into the tutorial, today we spent most of our time creating this notebook. But before day 3 we want to finish this section. Here it goes.

Let's create a node that connects to the loss, and has the capability of going through the connected variables and change them, optimizing the result:

In [8]:
# Optimizer.
optimizer = tf.train.GradientDescentOptimizer(0.02)
train = optimizer.minimize(loss)

We can now create a _training loop_ that will result in values for our variables that minimize the loss. Each time we run the train node we just created, the variables in our graph will hopefully converge more towards the best possible values.

In [9]:
# Training loop.
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})

# Results and the resulting value for the error function.
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s  b: %s  loss: %s" % (curr_W, curr_b, curr_loss))

W: [-1.]  b: [ 1.]  loss: 1.42871868951e-21


Next day, we will start with the [MNIST for beginners](https://www.tensorflow.org/get_started/mnist/beginners).

## Side notes on Day 2

It seems that the linear regression program can be done much simpler using the tf.contrib.learn package. Let's do it.

In [10]:
import tensorflow as tf
import numpy as np

In [11]:
# Declare list of features. We only have one real-valued feature. There are
# many other types of columns that are more complicated and useful.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

In [12]:
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# logistic regression, linear classification, logistic classification, and
# many neural network classifiers and regressors. The following code
# provides an estimator that does linear regression.
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f56e01d5668>, '_num_worker_replicas': 0, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_environment': 'local', '_task_id': 0, '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_secs': 600, '_model_dir': None, '_evaluation_master': ''}


In [13]:
# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train,
                                              batch_size=4,
                                              num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000)

In [14]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpm62ydyx3/model.ckpt.
INFO:tensorflow:loss = 6.5, step = 1
INFO:tensorflow:global_step/sec: 1436.39
INFO:tensorflow:loss = 0.0488201, step = 101 (0.071 sec)
INFO:tensorflow:global_step/sec: 1374.46
INFO:tensorflow:loss = 0.0396304, step = 201 (0.073 sec)
INFO:tensorflow:global_step/sec: 1188.59
INFO:tensorflow:loss = 0.00980587, step = 301 (0.084 sec)
INFO:tensorflow:global_step/sec: 1410.86
INFO:tensorflow:loss = 0.00252571, step = 401 (0.071 sec)
INFO:tensorflow:global_step/sec: 1456.57
INFO:tensorflow:loss = 0.00119089, step = 501 (0.069 sec)
INFO:tensorflow:global

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7f56e01d54a8>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [15]:
# Here we evaluate how well our model did.
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-24-01:47:11
INFO:tensorflow:Restoring parameters from /tmp/tmpm62ydyx3/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-24-01:47:11
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.68278e-06
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-24-01:47:11
I

And the results for the model evaluation

In [16]:
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

train loss: {'global_step': 1000, 'loss': 1.6827835e-06}
eval loss: {'global_step': 1000, 'loss': 0.0026557927}


The eval data has a higher loss, but it is still close to zero. That is supposed to mean that we are learning properly.

Finally, let's try creating a custom model that is not built into TensorFlow to implement our own equivalent model to LinearRegressor using the knowledge of the lower level TensorFlow API.

Instead of sub-classing tf.contrib.learn.Estimator, we provide Estimator, a function model_fn that tells tf.contrib.learn how it can evaluate predictions, training steps, and loss.

In [17]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f56a81d8390>, '_num_worker_replicas': 0, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_environment': 'local', '_task_id': 0, '_task_type': None, '_tf_random_seed': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_save_checkpoints_secs': 600, '_model_dir': None, '_evaluation_master': ''}


In [18]:
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)

In [19]:
# train
estimator.fit(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpb8l5gcul/model.ckpt.
INFO:tensorflow:loss = 99.2041000147, step = 1
INFO:tensorflow:global_step/sec: 1137.01
INFO:tensorflow:loss = 0.153499242501, step = 101 (0.090 sec)
INFO:tensorflow:global_step/sec: 1513.16
INFO:tensorflow:loss = 0.0140497510511, step = 201 (0.066 sec)
INFO:tensorflow:global_step/sec: 1367.18
INFO:tensorflow:loss = 0.00316732801752, step = 301 (0.073 sec)
INFO:tensorflow:global_step/sec: 1417.49
INFO:tensorflow:loss = 0.000142514244896, step = 401 (0.070 sec)
INFO:tensorflow:global_step/sec: 1503.55
INFO:tensorflow:loss = 2.55092471665e-05, step = 501 (0.066 sec)
INFO:tensorflow:global_step/sec: 1637.8
INFO:tensorflow:loss = 3.29001444816e-07, step = 601 (0.062 sec)
INFO:tensorflow:global_step/sec: 1335.64
INFO:tensorflow:loss = 8.98038434244e-08, step = 701 (0.074 sec)
INFO:tensorflow:global_step/sec: 1049.87
INFO:tensorflow:loss = 9.55069666317e-09, step = 801 (0.09

Estimator(params=None)

In [20]:
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Starting evaluation at 2017-06-24-01:47:14
INFO:tensorflow:Restoring parameters from /tmp/tmpb8l5gcul/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-24-01:47:14
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.42345e-10
INFO:tensorflow:Starting evaluation at 2017-06-24-01:47:14
INFO:tensorflow:Restoring parameters from /tmp/tmpb8l5gcul/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-06-24-01:47:15
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101027


And the results this time:

In [21]:
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

train loss: {'global_step': 1000, 'loss': 1.4234476e-10}
eval loss: {'global_step': 1000, 'loss': 0.010102653}


Boy, was that verbose.

## Day 3

We looked at MNIST today. Because this is a whole new session of TensorFlow, I think it makes sense to create a new notebook for it.